In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:

# 1. Cargar y limpiar datos
file_path = 'plastic_material_time_data_2023.csv'  # Ruta del archivo CSV
data = pd.read_csv(file_path)

In [ ]:

# Asegurarse de que la columna timestamp sea de tipo datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Crear una nueva columna con minutos desde el inicio (tiempo como variable numérica)
data['time_minutes'] = (data['timestamp'] - data['timestamp'].min()).dt.total_seconds() / 60

# Seleccionar solo las columnas necesarias para el modelo
cleaned_data = data[['time_minutes', 'material']]


In [ ]:

# 2. Escalar los datos
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(cleaned_data)

# Separar características (X) y etiquetas (y)
X = scaled_data[:, 0].reshape(-1, 1)  # Usamos solo el tiempo como entrada
y = scaled_data[:, 1]  # Usamos el material como salida

# Crear secuencias de datos para LSTM
def create_sequences(X, y, seq_length=10):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

seq_length = 10
X_seq, y_seq = create_sequences(X, y, seq_length)

In [ ]:
# 3. Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [ ]:

# 4. Construir la LSTM
model = Sequential([
    LSTM(50, activation='tanh', return_sequences=False, input_shape=(seq_length, 1)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# 5. Entrenar la LSTM
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# 6. Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
# 7. Hacer predicciones
predictions = model.predict(X_test)

In [ ]:
# 8. Convertir predicciones a valores binarios (0 o 1)
binary_predictions = (predictions > 0.5).astype(int)


In [ ]:
# Mostrar algunas predicciones
for i in range(10):
    print(f"Tiempo: {X_test[i].flatten()}, Predicción: {binary_predictions[i]}, Real: {y_test[i]}")